# Interactive Tutorial: Optical Coherence Tomography (OCT) Principles

This notebook provides an interactive introduction to the basic principles of Optical Coherence Tomography (OCT) through alternating theoretical explanations and practical simulations.

**Author**: OCT-Principle Educational Materials  
**License**: MIT  
**Compatible with**: Google Colab, Jupyter Notebook

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from IPython.display import display, HTML

# Set plot style for better visualization
# Set plot style (with fallback for older matplotlib versions)
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    try:
        plt.style.use('seaborn-darkgrid')
    except:
        plt.style.use('default')
%matplotlib inline

## 1. Theory: What is Optical Coherence Tomography?

**Optical Coherence Tomography (OCT)** is a non-invasive imaging technique that captures high-resolution cross-sectional images of biological tissues. Think of it as "ultrasound with light."

### Key Principles:

1. **Low-Coherence Interferometry**: OCT uses light with a short coherence length to measure the time delay of reflected light.
2. **Michelson Interferometer**: The basic OCT system is based on a Michelson interferometer that splits light into two paths:
   - **Reference arm**: Light travels a known distance
   - **Sample arm**: Light penetrates the tissue and reflects from different layers
3. **Interference Pattern**: When light from both arms recombines, it creates an interference pattern only when the path length difference is within the coherence length.

### Why Low-Coherence Light?

Low-coherence light (broad spectrum) enables precise depth resolution because interference only occurs when path lengths are closely matched, allowing us to distinguish between reflections from different tissue layers.

## 2. Simulation: Visualizing Coherence Length

Let's simulate how coherence length affects interference. We'll compare:
- **High-coherence light** (laser, narrow spectrum) → Long coherence length
- **Low-coherence light** (LED, broad spectrum) → Short coherence length

In [ ]:
# Parameters
lambda_0 = 850e-9  # Center wavelength (850 nm, typical for OCT)
c = 3e8  # Speed of light (m/s)

# Create path length difference array
delta_z = np.linspace(-50e-6, 50e-6, 1000)  # Path difference from -50 to +50 microns

# High-coherence light (narrow spectrum)
delta_lambda_narrow = 10e-9  # 10 nm bandwidth
coherence_length_narrow = lambda_0**2 / delta_lambda_narrow
interference_narrow = np.cos(2 * np.pi * delta_z / lambda_0) * np.exp(-(delta_z / coherence_length_narrow)**2)

# Low-coherence light (broad spectrum)
delta_lambda_broad = 50e-9  # 50 nm bandwidth (typical for OCT)
coherence_length_broad = lambda_0**2 / delta_lambda_broad
interference_broad = np.cos(2 * np.pi * delta_z / lambda_0) * np.exp(-(delta_z / coherence_length_broad)**2)

# Plot comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# High-coherence
ax1.plot(delta_z * 1e6, interference_narrow, 'b-', linewidth=1.5)
ax1.set_xlabel('Path Length Difference (μm)', fontsize=12)
ax1.set_ylabel('Interference Signal', fontsize=12)
ax1.set_title(f'High-Coherence Light\nCoherence Length ≈ {coherence_length_narrow*1e6:.1f} μm', fontsize=12)
ax1.grid(True, alpha=0.3)
ax1.axhline(y=0, color='k', linestyle='--', alpha=0.3)

# Low-coherence
ax2.plot(delta_z * 1e6, interference_broad, 'r-', linewidth=1.5)
ax2.set_xlabel('Path Length Difference (μm)', fontsize=12)
ax2.set_ylabel('Interference Signal', fontsize=12)
ax2.set_title(f'Low-Coherence Light (OCT)\nCoherence Length ≈ {coherence_length_broad*1e6:.1f} μm', fontsize=12)
ax2.grid(True, alpha=0.3)
ax2.axhline(y=0, color='k', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Key Observations:")
print(f"   • High-coherence light: Interference fringes extend over {coherence_length_narrow*1e6:.1f} μm")
print(f"   • Low-coherence light: Interference localized within {coherence_length_broad*1e6:.1f} μm")
print(f"   • Shorter coherence length → Better depth resolution for OCT!")

## 3. Theory: The Michelson Interferometer

The **Michelson interferometer** is the heart of OCT systems. Here's how it works:

### Components:

1. **Light Source**: Low-coherence light (superluminescent diode or femtosecond laser)
2. **Beam Splitter**: Divides light into two paths (50/50 split)
3. **Reference Mirror**: Reflects light back at a known distance
4. **Sample**: Tissue being imaged, with multiple reflecting layers
5. **Detector**: Measures the interference pattern

### How It Works:

1. Light from the source is split into reference and sample arms
2. Both beams reflect back and recombine at the beam splitter
3. **Constructive interference** occurs when: Path difference = nλ (n = integer)
4. **Destructive interference** occurs when: Path difference = (n + 0.5)λ
5. By scanning the reference mirror, we can map tissue structure at different depths

### Mathematical Expression:

The detected intensity is:

$$I_{det} = I_{ref} + I_{sample} + 2\sqrt{I_{ref} I_{sample}} \cos(2k\Delta z)$$

Where:
- $I_{ref}$: Reference arm intensity
- $I_{sample}$: Sample arm intensity  
- $k = 2\pi/\lambda$: Wave number
- $\Delta z$: Path length difference

## 4. Simulation: Michelson Interferometer Operation

Let's simulate an OCT measurement of a sample with multiple layers (like tissue with different structures).

In [ ]:
# Simulate a tissue sample with 3 layers at different depths
def simulate_oct_ascan(z_ref_scan, sample_layers, lambda_0=850e-9, delta_lambda=50e-9):
    """
    Simulate an OCT A-scan (depth scan)
    
    Parameters:
    - z_ref_scan: Array of reference mirror positions
    - sample_layers: List of (depth, reflectivity) tuples
    - lambda_0: Center wavelength
    - delta_lambda: Spectral bandwidth
    """
    coherence_length = lambda_0**2 / delta_lambda
    k0 = 2 * np.pi / lambda_0
    
    signal = np.zeros_like(z_ref_scan)
    
    # Each layer contributes to the signal when reference mirror matches its depth
    for z_layer, reflectivity in sample_layers:
        delta_z = z_ref_scan - z_layer
        # Interference signal: oscillation × envelope
        layer_signal = reflectivity * np.cos(k0 * delta_z) * np.exp(-(delta_z / coherence_length)**2)
        signal += layer_signal
    
    return signal

# Define sample: tissue with 3 layers
sample_layers = [
    (100e-6, 0.8),   # Surface layer at 100 μm, 80% reflectivity
    (250e-6, 0.5),   # Middle layer at 250 μm, 50% reflectivity
    (400e-6, 0.3),   # Deep layer at 400 μm, 30% reflectivity
]

# Scan reference mirror from 0 to 500 μm
z_ref = np.linspace(0, 500e-6, 2000)

# Simulate OCT signal
oct_signal = simulate_oct_ascan(z_ref, sample_layers)

# Plot the result
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Plot 1: Raw interferometric signal
ax1.plot(z_ref * 1e6, oct_signal, 'b-', linewidth=0.8)
ax1.set_xlabel('Reference Mirror Position (μm)', fontsize=12)
ax1.set_ylabel('Interference Signal', fontsize=12)
ax1.set_title('Raw OCT A-Scan (Interferometric Signal)', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Mark layer positions
for z_layer, refl in sample_layers:
    ax1.axvline(x=z_layer * 1e6, color='r', linestyle='--', alpha=0.5, linewidth=2)
    ax1.text(z_layer * 1e6, ax1.get_ylim()[1] * 0.9, f'{z_layer*1e6:.0f}μm\n{refl*100:.0f}%', 
             ha='center', fontsize=10, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Plot 2: Envelope (processed OCT image)
envelope = np.abs(signal.hilbert(oct_signal))
ax2.plot(z_ref * 1e6, envelope, 'r-', linewidth=2)
ax2.fill_between(z_ref * 1e6, envelope, alpha=0.3, color='red')
ax2.set_xlabel('Depth (μm)', fontsize=12)
ax2.set_ylabel('Reflectivity (A.U.)', fontsize=12)
ax2.set_title('Processed OCT A-Scan (Depth Profile)', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)

# Mark layer positions
for z_layer, refl in sample_layers:
    ax2.axvline(x=z_layer * 1e6, color='b', linestyle='--', alpha=0.5, linewidth=2)

plt.tight_layout()
plt.show()

print(f"\n🔬 Simulation Results:")
print(f"   • Three distinct tissue layers detected at:")
for i, (z, r) in enumerate(sample_layers, 1):
    print(f"     Layer {i}: {z*1e6:.0f} μm depth, {r*100:.0f}% reflectivity")
print(f"\n   • The envelope (bottom plot) shows the depth-resolved structure")
print(f"   • This is analogous to an ultrasound A-scan, but with much higher resolution!")

## 5. Theory: Axial Resolution in OCT

One of OCT's key advantages is its **high axial resolution** (depth resolution), which determines how well we can distinguish between two closely spaced reflectors.

### Resolution Formula:

The axial resolution ($\Delta z$) is determined by the coherence length:

$$\Delta z = \frac{2\ln(2)}{\pi} \cdot \frac{\lambda_0^2}{\Delta\lambda} \approx 0.44 \frac{\lambda_0^2}{\Delta\lambda}$$

Where:
- $\lambda_0$: Center wavelength of the light source
- $\Delta\lambda$: Spectral bandwidth (FWHM)

### Key Insights:

1. **Broader bandwidth → Better resolution**: A wider spectrum gives shorter coherence length
2. **Typical OCT resolution**: 1-15 μm (compared to ultrasound: 50-500 μm)
3. **Trade-off**: Broader bandwidth may reduce penetration depth due to scattering

### Example:

For $\lambda_0 = 850$ nm and $\Delta\lambda = 50$ nm:

$$\Delta z \approx 0.44 \times \frac{(850 \times 10^{-9})^2}{50 \times 10^{-9}} \approx 6.4 \text{ μm}$$

This is in tissue! (In air, divide by refractive index n ≈ 1.4)

## 6. Simulation: Effect of Bandwidth on Resolution

Let's explore how changing the light source bandwidth affects our ability to resolve closely spaced structures.

In [ ]:
# Two layers very close together
z_layer1 = 200e-6  # First layer at 200 μm
z_layer2 = 210e-6  # Second layer at 210 μm (10 μm apart)

close_layers = [
    (z_layer1, 0.7),
    (z_layer2, 0.6),
]

# Test different bandwidths
bandwidths = [20e-9, 50e-9, 100e-9]  # 20, 50, 100 nm
bandwidth_labels = ['20 nm (Poor)', '50 nm (Good)', '100 nm (Excellent)']

z_ref_fine = np.linspace(180e-6, 230e-6, 2000)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for i, (bw, label) in enumerate(zip(bandwidths, bandwidth_labels)):
    # Calculate resolution
    resolution = 0.44 * lambda_0**2 / bw
    
    # Simulate signal
    signal_raw = simulate_oct_ascan(z_ref_fine, close_layers, lambda_0=lambda_0, delta_lambda=bw)
    envelope = np.abs(signal.hilbert(signal_raw))
    
    # Plot
    ax = axes[i]
    ax.plot(z_ref_fine * 1e6, envelope, 'b-', linewidth=2)
    ax.fill_between(z_ref_fine * 1e6, envelope, alpha=0.3)
    ax.axvline(x=z_layer1 * 1e6, color='r', linestyle='--', alpha=0.6, linewidth=2, label='Layer 1')
    ax.axvline(x=z_layer2 * 1e6, color='g', linestyle='--', alpha=0.6, linewidth=2, label='Layer 2')
    ax.set_xlabel('Depth (μm)', fontsize=11)
    ax.set_ylabel('Signal Intensity', fontsize=11)
    ax.set_title(f'Bandwidth: {label}\nResolution: {resolution*1e6:.1f} μm', fontsize=11, fontweight='bold')
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)
    
    # Add annotation
    separation = (z_layer2 - z_layer1) * 1e6
    if resolution * 1e6 < separation:
        ax.text(0.5, 0.95, '✓ Layers Resolved', transform=ax.transAxes,
                ha='center', va='top', fontsize=10, color='green',
                bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))
    else:
        ax.text(0.5, 0.95, '✗ Layers Not Resolved', transform=ax.transAxes,
                ha='center', va='top', fontsize=10, color='red',
                bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.7))

plt.tight_layout()
plt.show()

print(f"\n📏 Resolution Analysis:")
print(f"   • Layer separation: {(z_layer2 - z_layer1)*1e6:.1f} μm")
print(f"\n   Bandwidth vs Resolution:")
for bw, label in zip(bandwidths, bandwidth_labels):
    res = 0.44 * lambda_0**2 / bw
    print(f"   • {bw*1e9:.0f} nm → Resolution: {res*1e6:.1f} μm")
print(f"\n   💡 Conclusion: Wider bandwidth = Better resolution!")

## 7. Theory: Building a 2D OCT Image (B-Scan)

So far, we've looked at **A-scans** (1D depth profiles). To create a 2D cross-sectional image (**B-scan**), we need to:

### Process:

1. **Acquire A-scan**: Measure depth profile at one lateral position
2. **Lateral scanning**: Move the beam to a new position
3. **Repeat**: Acquire another A-scan
4. **Concatenate**: Stack all A-scans side-by-side to form a 2D image

### Image Dimensions:

- **Vertical axis**: Depth (determined by axial resolution)
- **Horizontal axis**: Lateral position (determined by beam focusing)
- **Pixel intensity**: Reflectivity of tissue at that location

### Scanning Methods:

1. **Time-Domain OCT (TD-OCT)**: Mechanically scan reference mirror for each A-scan (slower)
2. **Fourier-Domain OCT (FD-OCT)**: Capture full depth info simultaneously (faster)
3. **Swept-Source OCT (SS-OCT)**: Use tunable laser for high-speed imaging

## 8. Simulation: Creating a 2D OCT Image

Let's simulate a complete B-scan of a layered tissue structure with some variation.

In [ ]:
# Create a synthetic tissue structure with varying layer depths
def create_tissue_structure(n_ascans=200, n_depth=500):
    """
    Create a synthetic tissue structure with 3 layers
    """
    image = np.zeros((n_depth, n_ascans))
    x_positions = np.linspace(0, 1, n_ascans)
    
    # Layer 1: Surface (with slight curvature)
    surface_depth = 50 + 20 * np.sin(2 * np.pi * x_positions)
    surface_depth = surface_depth.astype(int)
    
    # Layer 2: Middle layer (with undulation)
    middle_depth = 200 + 30 * np.sin(4 * np.pi * x_positions) + 10 * np.cos(8 * np.pi * x_positions)
    middle_depth = middle_depth.astype(int)
    
    # Layer 3: Deep layer (relatively flat)
    deep_depth = 350 + 15 * np.sin(3 * np.pi * x_positions)
    deep_depth = deep_depth.astype(int)
    
    # Add layers to image with realistic OCT characteristics
    for i, x_pos in enumerate(range(n_ascans)):
        # Surface layer (strong reflection)
        if 0 <= surface_depth[i] < n_depth:
            # Add point spread function
            for offset in range(-3, 4):
                depth_idx = surface_depth[i] + offset
                if 0 <= depth_idx < n_depth:
                    image[depth_idx, x_pos] += 0.9 * np.exp(-offset**2 / 2)
        
        # Middle layer (medium reflection)
        if 0 <= middle_depth[i] < n_depth:
            for offset in range(-3, 4):
                depth_idx = middle_depth[i] + offset
                if 0 <= depth_idx < n_depth:
                    image[depth_idx, x_pos] += 0.6 * np.exp(-offset**2 / 2)
        
        # Deep layer (weak reflection due to attenuation)
        if 0 <= deep_depth[i] < n_depth:
            for offset in range(-3, 4):
                depth_idx = deep_depth[i] + offset
                if 0 <= depth_idx < n_depth:
                    image[depth_idx, x_pos] += 0.35 * np.exp(-offset**2 / 2)
        
        # Add attenuation with depth
        attenuation = np.exp(-np.arange(n_depth) / 200)
        image[:, x_pos] *= attenuation
        
        # Add speckle noise (characteristic of OCT)
        image[:, x_pos] += np.random.rayleigh(0.05, n_depth)
    
    return image

# Generate synthetic OCT B-scan
oct_image = create_tissue_structure(n_ascans=300, n_depth=500)

# Display the OCT image
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Linear scale
im1 = ax1.imshow(oct_image, aspect='auto', cmap='gray', extent=[0, 300, 500, 0])
ax1.set_xlabel('Lateral Position (A-scan number)', fontsize=12)
ax1.set_ylabel('Depth (pixels)', fontsize=12)
ax1.set_title('OCT B-Scan (Linear Scale)', fontsize=13, fontweight='bold')
plt.colorbar(im1, ax=ax1, label='Intensity')

# Logarithmic scale (common in OCT to enhance weak signals)
oct_image_log = 20 * np.log10(oct_image + 1e-10)  # Convert to dB
im2 = ax2.imshow(oct_image_log, aspect='auto', cmap='gray', vmin=-20, vmax=0, extent=[0, 300, 500, 0])
ax2.set_xlabel('Lateral Position (A-scan number)', fontsize=12)
ax2.set_ylabel('Depth (pixels)', fontsize=12)
ax2.set_title('OCT B-Scan (Logarithmic Scale, dB)', fontsize=13, fontweight='bold')
plt.colorbar(im2, ax=ax2, label='Intensity (dB)')

plt.tight_layout()
plt.show()

print(f"\n🖼️ OCT B-Scan Generated:")
print(f"   • Image size: {oct_image.shape[1]} A-scans × {oct_image.shape[0]} depth pixels")
print(f"   • Three distinct tissue layers visible")
print(f"   • Logarithmic scale (right) enhances visualization of deeper structures")
print(f"   • Speckle pattern is characteristic of coherent imaging systems")
print(f"\n   💡 This is similar to what ophthalmologists see when imaging the retina!")

## 9. Summary and Key Takeaways

### What We've Learned:

1. **OCT Principle**: 
   - Uses low-coherence interferometry to create high-resolution depth images
   - Based on Michelson interferometer design

2. **Coherence Length**:
   - Low-coherence light has short coherence length
   - Enables precise depth localization

3. **Resolution**:
   - Axial resolution ∝ λ₀²/Δλ
   - Broader bandwidth → Better resolution
   - Typical OCT: 1-15 μm resolution

4. **Image Formation**:
   - A-scan: 1D depth profile
   - B-scan: 2D cross-sectional image (stack of A-scans)
   - C-scan: 3D volumetric imaging

### Clinical Applications:

- **Ophthalmology**: Retinal imaging (most common use)
- **Cardiology**: Coronary artery imaging
- **Dermatology**: Skin lesion assessment
- **Oncology**: Early cancer detection

### Further Exploration:

- Fourier-Domain OCT (faster imaging)
- Doppler OCT (blood flow measurement)
- Polarization-sensitive OCT (tissue birefringence)
- OCT Angiography (OCTA)

## 10. Interactive Exercise: Design Your Own OCT System

Now it's your turn! Modify the parameters below to explore how different design choices affect OCT performance.

In [ ]:
# Interactive parameter exploration
print("🔧 OCT System Design Parameters:\n")
print("Modify these values and re-run to see the effects:\n")

# User-adjustable parameters
center_wavelength_nm = 850  # Try: 800, 850, 1310 nm
bandwidth_nm = 50            # Try: 20, 50, 100, 150 nm

# Calculate performance metrics
lambda_0 = center_wavelength_nm * 1e-9
delta_lambda = bandwidth_nm * 1e-9

coherence_length = lambda_0**2 / delta_lambda
axial_resolution = 0.44 * coherence_length
n_tissue = 1.4  # Refractive index of tissue
axial_resolution_tissue = axial_resolution / n_tissue

# Display results
print(f"\n📊 System Performance:")
print(f"   Center Wavelength: {center_wavelength_nm} nm")
print(f"   Bandwidth: {bandwidth_nm} nm")
print(f"   ─────────────────────────────────────")
print(f"   Coherence Length: {coherence_length*1e6:.2f} μm")
print(f"   Axial Resolution (in air): {axial_resolution*1e6:.2f} μm")
print(f"   Axial Resolution (in tissue): {axial_resolution_tissue*1e6:.2f} μm")

# Visual representation
fig, ax = plt.subplots(figsize=(10, 6))

# Create sample layers
test_layers = [(100e-6, 0.8), (150e-6, 0.6)]
z_test = np.linspace(50e-6, 200e-6, 1000)
test_signal = simulate_oct_ascan(z_test, test_layers, lambda_0, delta_lambda)
test_envelope = np.abs(signal.hilbert(test_signal))

ax.plot(z_test * 1e6, test_envelope, 'b-', linewidth=2)
ax.fill_between(z_test * 1e6, test_envelope, alpha=0.3)
ax.axvline(x=100, color='r', linestyle='--', alpha=0.6, linewidth=2)
ax.axvline(x=150, color='r', linestyle='--', alpha=0.6, linewidth=2)
ax.set_xlabel('Depth (μm)', fontsize=12)
ax.set_ylabel('Signal Intensity', fontsize=12)
ax.set_title(f'OCT Performance: {center_wavelength_nm}nm, Δλ={bandwidth_nm}nm\nResolution: {axial_resolution_tissue*1e6:.2f} μm', 
             fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Performance assessment
if axial_resolution_tissue*1e6 < 5:
    print(f"\n   ✓ Excellent resolution for cellular imaging!")
elif axial_resolution_tissue*1e6 < 10:
    print(f"\n   ✓ Good resolution for tissue structure imaging")
elif axial_resolution_tissue*1e6 < 20:
    print(f"\n   ⚠ Moderate resolution, suitable for large structures")
else:
    print(f"\n   ⚠ Low resolution, consider increasing bandwidth")

## References and Further Reading

1. Huang, D., et al. (1991). "Optical coherence tomography." *Science*, 254(5035), 1178-1181.
2. Drexler, W., & Fujimoto, J. G. (2008). *Optical coherence tomography: technology and applications*. Springer Science & Business Media.
3. Fercher, A. F., et al. (2003). "Optical coherence tomography-principles and applications." *Reports on Progress in Physics*, 66(2), 239.
4. Fujimoto, J., & Swanson, E. (2016). "The development, commercialization, and impact of optical coherence tomography." *Investigative Ophthalmology & Visual Science*, 57(9), OCT1-OCT13.

---

**📝 Note**: This notebook is designed for educational purposes. For clinical OCT imaging, specialized equipment and trained professionals are required.

**💻 GitHub Repository**: [TheBeatzzz/OCT-Principle](https://github.com/TheBeatzzz/OCT-Principle)

**📧 Questions or Feedback**: Please open an issue on the GitHub repository.